In [2]:
import os
import numpy as np
!pip install -q wfdb
import matplotlib.pyplot as plt
import wfdb  # MIT-BIH compatible reader
from scipy.signal import butter, filtfilt, iirnotch
from sklearn.model_selection import train_test_split

In [2]:
# dataset constants
NUM_OF_PARTICIPANTS = 43
NUM_OF_SESSIONS = 3
NUM_OF_GESTURES = 16
NUM_OF_TRIALS = 7
FS = 2048

In [3]:
from scipy.signal import butter, filtfilt, iirnotch

"""Bandpass filter to keep only EMG band."""
def _bandpass_filter(data, low=20, high=450, fs=2048, order=4):
    nyq = fs / 2
    b, a = butter(order, [low / nyq, high / nyq], btype='band')
    return filtfilt(b, a, data, axis=0)

"""Notch filter to remove powerline noise (60Hz)."""
def _notch_filter(data, freq=60.0, fs=2048, quality=30):
    """Notch filter to remove powerline noise (60 Hz)."""
    nyq = fs / 2
    w0 = freq / nyq
    b, a = iirnotch(w0, quality)
    return filtfilt(b, a, data, axis=0)

"""High-pass filter to remove DC offset / drift."""
def _dc_removal(data, cutoff=0.1, fs=2048, order=2):
    nyq = fs / 2
    b, a = butter(order, cutoff / nyq, btype='high')
    return filtfilt(b, a, data, axis=0)

"""Apply DC removal + 60Hz notch + bandpass in order."""
def full_preprocess(data, fs=2048):
    x = _dc_removal(data, fs=fs)
    x = _notch_filter(x, fs=fs)
    x = _bandpass_filter(x, fs=fs)
    return x

In [4]:
mainFolder = r".\\Grabmyo-1.0.2"
converted_root = "Processed_data"
os.makedirs(converted_root, exist_ok=True)  # dir for the processed + windowed data

FOREARM_CHANNELS = list(range(0, 16))
WRIST_CHANNELS = list(range(17, 23)) + list(range(25, 31))

count = 0
for sessionNum in range(1, NUM_OF_SESSIONS + 1):
    sessionFolder = f"Session{sessionNum}_Converted"
    sessionPath = os.path.join(converted_root, sessionFolder)
    os.makedirs(sessionPath, exist_ok=True)

    for participantNum in range(1, NUM_OF_PARTICIPANTS + 1):
        forearmData = {}  # dict: (gesture,trial) -> ndarray (samples x 16)
        wristData   = {}  # dict: (gesture,trial) -> ndarray (samples x 12)

        participantFolder = f"session{sessionNum}_participant{participantNum}"
        for gestureNum in range(1, NUM_OF_GESTURES + 2):  # +1 for resting -> 17 labels
            for trialNum in range(1, NUM_OF_TRIALS + 1):
                filename = f"session{sessionNum}_participant{participantNum}_gesture{gestureNum}_trial{trialNum}"
                filepath = os.path.join(mainFolder, f"Session{sessionNum}", participantFolder, filename)

                try:
                    record = wfdb.rdrecord(filepath)
                    emgData = record.p_signal  # shape (n_samples, n_channels)
                except Exception as e:
                    print(f"[WARN] Error reading {filepath}: {e}")
                    continue

                # Select channels and preprocess separately
                try:
                    forearm = emgData[:, FOREARM_CHANNELS]  # (n_samples, 16)
                    wrist   = emgData[:, WRIST_CHANNELS]    # (n_samples, 12)
                except Exception as e:
                    print(f"[WARN] Channel indexing problem for {filepath}: {e}")
                    continue

                forearm_pp = full_preprocess(forearm, fs=FS)
                wrist_pp   = full_preprocess(wrist, fs=FS)

                # Save under keys (gesture, trial) - consistent ordering
                forearmData[(gestureNum, trialNum)] = forearm_pp
                wristData[(gestureNum, trialNum)] = wrist_pp

        # Save per-participant file (preprocessed, un-windowed)
        save_path = os.path.join(sessionPath, f"session{sessionNum}_participant{participantNum}.npz")
        # storing dicts is fine (they are pickled inside npz); keeps structure for on-the-fly windowing
        np.savez_compressed(save_path, forearmData=forearmData, wristData=wristData)

        count += 1
        print(f"Converted: {count} of {NUM_OF_PARTICIPANTS * NUM_OF_SESSIONS} participants")

print("Participant-level conversion + preprocessing (no windowing) completed.")

Converted: 1 of 129 participants
Converted: 2 of 129 participants
Converted: 3 of 129 participants
Converted: 4 of 129 participants
Converted: 5 of 129 participants
Converted: 6 of 129 participants
Converted: 7 of 129 participants
Converted: 8 of 129 participants
Converted: 9 of 129 participants
Converted: 10 of 129 participants
Converted: 11 of 129 participants
Converted: 12 of 129 participants
Converted: 13 of 129 participants
Converted: 14 of 129 participants
Converted: 15 of 129 participants
Converted: 16 of 129 participants
Converted: 17 of 129 participants
Converted: 18 of 129 participants
Converted: 19 of 129 participants
Converted: 20 of 129 participants
Converted: 21 of 129 participants
Converted: 22 of 129 participants
Converted: 23 of 129 participants
Converted: 24 of 129 participants
Converted: 25 of 129 participants
Converted: 26 of 129 participants
Converted: 27 of 129 participants
Converted: 28 of 129 participants
Converted: 29 of 129 participants
Converted: 30 of 129 pa

In [3]:
sample = np.load(r".\Processed_data\Session1_Converted\session1_participant36.npz", allow_pickle=True)
data = sample['forearmData'].item()[(1,1)]  # gesture 1, trial 1
print(data.shape)
print(data)

(10240, 16)
[[ 0.00082468  0.00103293  0.00181015 ...  0.00078637  0.00120205
   0.00115372]
 [-0.00349553 -0.00424119  0.00209671 ...  0.00094024 -0.00765628
  -0.00499999]
 [-0.0047394  -0.00697291  0.00224367 ...  0.00445375 -0.01070339
  -0.00771597]
 ...
 [-0.02658384 -0.02291809 -0.03653183 ... -0.01205873 -0.0282834
  -0.01065593]
 [-0.02263637 -0.0243561  -0.04046352 ... -0.01629123 -0.0243865
  -0.01068911]
 [-0.00376234 -0.01069089 -0.0231561  ... -0.00555721 -0.00516356
   0.00379449]]
